In [ ]:
'''
    로지스틱 회귀(logistic regression) - 이진분류(binary classification)
    - 선형회귀의 선형방정식을 이용해서 분류 알고리즘으로 사용
    - 선형방정식에서 나온 값을 sigmoid 함수를 통해서 0~1 사이로 압축해서 확률로 간주하고 처리한다.
    

'''

In [1]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv(r"C:\Users\YB\Desktop\tf24_study\csv\fish.csv")
df

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...,...
154,Smelt,12.2,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,14.3,15.2,2.8728,2.0672


In [5]:
df['Species'].value_counts()

Perch        56
Bream        35
Roach        20
Pike         17
Smelt        14
Parkki       11
Whitefish     6
Name: Species, dtype: int64

### 2. 입력데이터와 label 작성

In [8]:
fish_data = df.iloc[:,1:]
fish_target = df.iloc[:,0]

In [9]:
fish_data.shape, fish_target.shape

((159, 5), (159,))

### 3. 훈련데이터와 테스트 데이터 분리

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test=train_test_split(fish_data, fish_target, test_size=0.2, random_state=1,stratify=fish_target)

In [13]:
### 총 159개 중에 127개가 훈련 데이터
X_train.shape,y_train.shape

((127, 5), (127,))

In [15]:
### 총 159개 중에 32개가 테스트 데이터
X_test.shape, y_test.shape

((32, 5), (32,))

### 4. 표준화

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
sc = StandardScaler()
sc.fit(X_train)
X_train_scaled = sc.transform(X_train)
X_test_scaled = sc.transform(X_test)

### 이진분류이기 때문에 7종류에서 Bream 와 Smelt만 추출
* fish_target에서 Bream 또는 Smelt에 해당하는 index 값을 추출 
* 추출된 index 이용해서 실제 데이터 추출한다. 

In [23]:
bream_smelt_indexs = (y_train == 'Bream') | (y_train == 'Smelt')

In [40]:
# 훈련 데이터
X_train_bream_smelt = X_train_scaled[bream_smelt_indexs]
# 훈련 데이터 label
y_train_bream_smelt = y_train[bream_smelt_indexs]
X_train_bream_smelt.shape, y_train_bream_smelt.shape

((39, 5), (39,))

In [31]:
bream_smelt_indexs2 = (y_test == 'Bream') | (y_test == 'Smelt')

In [32]:
# 테스트 데이터
X_test_bream_smelt = X_test_scaled[bream_smelt_indexs2]
# 테스트 데이터 label
y_test_bream_smelt = y_test[bream_smelt_indexs2]

In [36]:
X_test_bream_smelt.shape,y_test_bream_smelt.shape

((10, 5), (10,))

### 5. 모델 생성

In [37]:
from sklearn.linear_model import LogisticRegression

In [43]:
lr = LogisticRegression(C=1.0,random_state=42) # C 기능은 Ridge의 alpha 와 동일한 기능이다. C는 작을수록 규제 강도가 강해진다.
lr.fit(X_train_bream_smelt,y_train_bream_smelt)

LogisticRegression(random_state=42)

### 6. 예측

In [45]:
pred = lr.predict(X_test_bream_smelt)
pred # 새 데이터를 넣어줬더니 예측함

array(['Bream', 'Bream', 'Bream', 'Bream', 'Smelt', 'Smelt', 'Bream',
       'Bream', 'Smelt', 'Bream'], dtype=object)

### 예측된 과정

### 가. label 종류 - 클래스 확인 
* [Bream, Smelt] ==> [음성, 양성] ==> [1-양성, 양성]

In [46]:
lr.classes_

array(['Bream', 'Smelt'], dtype=object)

### 나. 회귀계수 확인 

In [50]:
fish_data.columns

Index(['Weight', 'Length', 'Diagonal', 'Height', 'Width'], dtype='object')

In [51]:
lr.coef_, lr.intercept_ #feature가 5개여서 계수가 5개가 나옴b

(array([[-0.43861356, -0.61493657, -0.69159991, -0.97184449, -0.78356712]]),
 array([-2.19442896]))

### 선형 방정식
* y = w1*x1 + w2*x2 + w3*x3 + w4*x4 + w5*x5 + w0
* y = -0.43 * Weight + -0.61 * Length + -0.69 * Diagonal + -0.97 * Height + -0.78 * Width + -2.19

### 실제 5개의 feature 값을 위의 선형 회귀식에 직접 대입 하지 않고 y값을 구해주는 함수를 사용한다.
* lr.decision_function(데이터)

In [54]:
X_test_bream_smelt

array([[ 0.5060019 ,  0.29981453,  0.47712474,  1.44669788,  0.6339306 ],
       [ 0.09690193,  0.11488765,  0.29807709,  1.11934743,  0.20430551],
       [ 0.09690193,  0.08714861,  0.26397277,  1.02740789,  0.25309206],
       [ 0.23326859,  0.17961206,  0.39186395,  1.16998841,  0.27176634],
       [-1.1036701 , -1.54945432, -1.56913417, -1.57613717, -1.95222339],
       [-1.0971245 , -1.53096163, -1.55208201, -1.60217719, -1.80913174],
       [-0.47038335, -0.31044419, -0.13675293,  0.55174354, -0.27644042],
       [ 0.23326859,  0.25358281,  0.40891611,  1.06322433, -0.07335765],
       [-1.0766695 , -1.3367884 , -1.39861259, -1.42346042, -1.4160382 ],
       [ 0.23326859,  0.20735109,  0.39186395,  1.2030638 ,  0.18726523]])

In [55]:
decisions = lr.decision_function(X_test_bream_smelt)
decisions

array([-4.83340713, -3.76164887, -3.66988118, -4.02820078,  3.38914316,
        3.27629201, -2.02222993, -3.71129548,  2.56007645, -4.01119037])

In [56]:
### decision_function 대신에 직접 계산 
y = -0.43 *  0.5060019 + -0.61 *  0.29981453 + -0.69 * 0.47712474 + -0.97 * 1.44669788 + -0.78 *  + -2.19
y


-0.4247806945000001

### y값을 sigmoid 함수에 적용한다.
* 양성확률 값을 반환
* [음성, 양성]  음성확률이란 not~일 확률

In [58]:
from scipy.special import expit

In [59]:
expit(decisions) 

array([0.00789651, 0.02271731, 0.02484642, 0.01749482, 0.9673635 ,
       0.96360647, 0.11688861, 0.0238625 , 0.92824755, 0.01778962])

In [60]:
### 수동
[1-0.00789651,0.00789651]

[0.99210349, 0.00789651]

### 전체 예측 확률 값으로 보기
* lr.classes_ [Bream, Smelt]

In [63]:
#array(['Bream', 'Bream', 'Bream', 'Bream', 'Smelt', 'Smelt', 'Bream','Bream', 'Smelt', 'Bream'], dtype=object)
lr.predict_proba(X_test_bream_smelt)

array([[0.99210349, 0.00789651],
       [0.97728269, 0.02271731],
       [0.97515358, 0.02484642],
       [0.98250518, 0.01749482],
       [0.0326365 , 0.9673635 ],
       [0.03639353, 0.96360647],
       [0.88311139, 0.11688861],
       [0.9761375 , 0.0238625 ],
       [0.07175245, 0.92824755],
       [0.98221038, 0.01778962]])